In [1]:
# sql functions
from sqlalchemy import create_engine
import psycopg2

import pandas as pd

import pickle

dbname = 'birth_db'
username = 'shengsql'
pswd = '1234'
con = None
con = psycopg2.connect(database = dbname, user = username, host='localhost', password=pswd)
cursor = con.cursor()

## query all project information

In [2]:
# query all project information, joining project metadata with storis info:
sql_query = """SELECT *
    FROM stories LEFT JOIN projects ON (stories.project_id = projects.project_id);"""

project_data_from_sql = pd.read_sql_query(sql_query,con).drop(columns=['index']).drop_duplicates()

# Filter out non-english entries and ones without a story

In [3]:
# make function to detect langauge:
def trydetect(s):
    try:
        r = detect(s)
    except:
        r = "NA"
    return r

In [4]:
from langdetect import detect
# detect the language of the blurb
project_data_from_sql['language'] = project_data_from_sql.apply(lambda x: trydetect(x['blurb']), axis=1)
# decide if a proposals really has a story section, some had them in pictures thus not in my data
project_data_from_sql['keep_story'] = project_data_from_sql.apply(lambda x: 
                                                                  len(x['story']) > len(x['blurb']), axis=1)

In [5]:
# filter out non-english entries and entries without much stories
data_project = project_data_from_sql[(project_data_from_sql.keep_story == 1) & 
                                     (project_data_from_sql.language == "en")].reset_index()
data_project.shape

(2107, 30)

In [6]:
pickle.dump(data_project, open("pickle/project_data_complete.pkl", "wb"))